# Basic imports and utils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import layer
from neurons import RBF
from typing import List
import loss

## Constants

# RBF


We choose a gaußian activation function for a RBF neuron with it's derivatives $$y(x)=e^{-\frac{||x - c||^2}{2\sigma^2}}$$
$$\frac d{dc}y(x)=\frac{x-c}{\sigma^2}e^{-\frac{||x - c||^2}{2\sigma^2}}$$
$$\frac d{dx}y(x)=-\frac{x-c}{\sigma^2}e^{-\frac{||x - c||^2}{2\sigma^2}}$$

The standard deviation is going to be a fixed size hyper parameter.
## Definition of an RBF in this module
To use the broadcasting abilities of numpy as much as possible, we design functions in a way that they can update multiple neurons at once. For this we design the data structures for our neurons in a data oriented way. For this to work, the only limit put onto our neurons is that the input dimensions should be the same for all neurons.

Since we are going to work on images, and an image is a $(w, h, 3)$ float or integer array an RBF neuron needs to define the following things.

c: Should be a $(k_w, k_h, k_c)$ array representing the centers for this neuron

x: A subimage that should be defined by the outside function with the same dimesnions as c.

# Layer

# Experiments


In [ ]:
TRAIN_RATE:float = 0.1
TRAIN:bool = True
FIRST_LAYER_NEURONS:int = 3
TRAINING_REPITIONS:int = 100

layers:List[layer.DenseLayer] = []
layers.append(layer.DenseLayer(2, FIRST_LAYER_NEURONS, RBF))
layers.append(layer.DenseLayer(FIRST_LAYER_NEURONS, 1, RBF))

#layers[0].neuron_data = np.array([[0.0,0.0], [1.0,0.0], [0.0,1.0]])
#layers[1].neuron_data = np.array([[0.0,0.0,0.0]])

inputs = np.array([[0.0,0.0],[1.0,0.0],[0.0,1.0],[1.0,1.0]])
expected = np.array([0.0,1.0,1.0,1.0])

losses = []
errors = []

if TRAIN:
    for _ in range(TRAINING_REPITIONS):
        order = np.arange(4)
        np.random.shuffle(order)
        for i in range(4):
            current_input = inputs[order[i]]
            for l in layers:
                current_input = l.propagate(current_input)
            errors.append(np.abs(expected[order[i]] - current_input[0]))
            loss_i, deriv = loss.quadratic_error(current_input[0], expected[order[i]])
            losses.append(loss_i)

            deriv = layers[1].back_propagate(deriv, TRAIN_RATE)
            deriv = layers[0].back_propagate(deriv, TRAIN_RATE)

    plt.plot(errors, label='Error')
    plt.plot(losses, label='Loss')
    plt.legend()

for i in range(4):
    current_input = inputs[i]
    for l in layers:
        current_input = l.propagate(current_input)
    print(f"Output: {current_input[0]}\tExpected: {expected[i]}")

pts = np.empty((FIRST_LAYER_NEURONS, 2))
colours = np.empty((FIRST_LAYER_NEURONS,3))
pts = layers[0].neuron_data
colours[:] = np.array([1.0,0.0,0.0])
plt.figure()
plt.scatter(pts[:,0], pts[:,1],c=colours)


